In [69]:
import torch
from pytorch_memlab import LineProfiler
import pandas as pd
import logging
import warnings
warnings.filterwarnings('ignore')

def outer():
    linear = torch.nn.Linear(100, 100).cuda()
    linear2 = torch.nn.Linear(100, 100).cuda()
    linear3 = torch.nn.Linear(100, 100).cuda()


with LineProfiler(outer) as prof:
    outer()
    
    
def inner():
    linear = torch.nn.Linear(200, 200).cuda()
    linear2 = torch.nn.Linear(200, 200).cuda()
    linear3 = torch.nn.Linear(200, 200).cuda()


with LineProfiler(inner) as prof2:
    inner()


data_frame_outer = pd.read_html(prof.display()._repr_html_())[0]
data_frame_outer.columns = [i[0] for i in data_frame_outer.columns]

data_frame_inner = pd.read_html(prof2.display()._repr_html_())[0]
data_frame_inner.columns = [i[0] for i in data_frame_inner.columns]

def rearrange_df(df1, df2, batch_size):
    
    new_df = pd.DataFrame()
    
    active1 = df1['active_bytes']
    active2 = df2['active_bytes']
    reserved1 = df1['reserved_bytes']
    reserved2 = df2['reserved_bytes']
    for i in range(len(active1)):
        if active1[i][-1] == 'G':
            active1[i] = float(active1[i][:-1]) * 1024
            reserved1[i] = float(reserved1[i][:-1]) * 1024
        if active1[i][-1] == 'K':
            active1[i] = float(active1[i][:-1]) / 1024
            if reserved1[i][-1] == 'K':
                reserved1[i] = float(reserved1[i][:-1]) / 1024
            else:
                reserved1[i] = float(reserved1[i][:-1])
        else:
            active1[i] = float(active1[i][:-1])
            reserved1[i] = float(reserved1[i][:-1])
    
    for i in range(len(active2)):
        if active2[i][-1] == 'G':
            active2[i] = float(active2[i][:-1]) * 1024
            reserved2[i] = float(reserved2[i][:-1]) * 1024
        if active2[i][-1] == 'K':
            active2[i] = float(active2[i][:-1]) / 1024
            if reserved2[i][-1] == 'K':
                reserved2[i] = float(reserved2[i][:-1]) / 1024
            else:
                reserved2[i] = float(reserved2[i][:-1])
        else:
            active2[i] = float(active2[i][:-1])
            reserved2[i] = float(reserved2[i][:-1])
    
    new_df['active_bytes'] = (active1 - active2) / (batch_size - 1)
    new_df['reserved_bytes'] = (reserved1 - reserved2) / (batch_size - 1)
    new_df = new_df.astype('float').round(3)
    new_df = new_df.astype('str') + 'M'
    return new_df

In [14]:
data_frame_outer

,active_bytes,reserved_bytes,line,code
0,0.00B,0.00B,7,def outer():
1,40.00K,2.00M,8,"linear = torch.nn.Linear(100, 100).cuda()"
2,80.00K,2.00M,9,"linear2 = torch.nn.Linear(100, 100).cuda()"
3,120.00K,2.00M,10,"linear3 = torch.nn.Linear(100, 100).cuda()"


In [15]:
data_frame_inner

,active_bytes,reserved_bytes,line,code
0,0.00B,0.00B,17,def inner():
1,157.50K,2.00M,18,"linear = torch.nn.Linear(200, 200).cuda()"
2,315.00K,2.00M,19,"linear2 = torch.nn.Linear(200, 200).cuda()"
3,472.50K,2.00M,20,"linear3 = torch.nn.Linear(200, 200).cuda()"


In [70]:
s = rearrange_df(data_frame_inner, data_frame_outer, 2)
s

,active_bytes,reserved_bytes
0,0.0M,0.0M
1,0.115M,0.0M
2,0.229M,0.0M
3,0.344M,0.0M


In [18]:
a = pd.Series([1, 2])
b = pd.Series([3, 4])
(b - a) / 2

0    1.0
1    1.0
dtype: float64

[1]